In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re

In [2]:
fakeid_accn = pd.read_csv("/data/UCSD_cxr/falseids-accn.csv").drop_duplicates()

In [3]:
# Get unique patients
unq_fakeid = fakeid_accn['FalseID'].unique()
num_unq_fake = unq_fakeid.size
np.random.seed(1)

In [4]:
# Split dataset
shuffled_ids = np.random.choice(unq_fakeid, size=num_unq_fake, replace=False)
train_size = 0.98
split = int(train_size*num_unq_fake)
train = shuffled_ids[:split]
test = shuffled_ids[split:]

In [5]:
train_accn = fakeid_accn[fakeid_accn['FalseID'].isin(train)]
test_accn = fakeid_accn[fakeid_accn['FalseID'].isin(test)]
train_accn.to_csv('/data/UCSD_cxr/train_accn.csv')
test_accn.to_csv('/data/UCSD_cxr/test_accn.csv')

In [6]:
fakeid_accn.drop_duplicates().shape[0] == (train_accn.shape[0] + test_accn.shape[0])

True

In [7]:
rep_t2020 = pd.read_csv('/data/UCSD_cxr/all-reports-available-xrays-through2020.csv').dropna().drop_duplicates()
rep_a2020 = pd.read_csv('/data/UCSD_cxr/all-reports-available-xrays-after2020.csv').dropna().drop_duplicates()
rep_a2020['AccessionId'] = rep_a2020['AccessionId'].map(str)

/tmp/ipykernel_308396/215930045.py:1: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  rep_t2020 = pd.read_csv('/data/UCSD_cxr/all-reports-available-xrays-through2020.csv').dropna().drop_duplicates()


In [8]:
train_rep_t2020 = train_accn.merge(rep_t2020, left_on="Accession", right_on="AccessionId")

In [21]:
test_accn

,FalseID,Accession
93,28369992,53321530
96,28422472,50388616
98,28447808,54129354
99,28447808,50001322
100,28447808,52403905
...,...,...
188008,258673824,56932992
188095,426514600,52483773
188096,426533176,53517031
188149,472099952,54323529


In [9]:
test_rep_t2020 = test_accn.merge(rep_t2020, left_on="Accession", right_on="AccessionId")

In [10]:
train_rep_a2020 = train_accn.merge(rep_a2020, left_on="Accession", right_on="AccessionId")

In [11]:
test_rep_a2020 = test_accn.merge(rep_a2020, left_on="Accession", right_on="AccessionId")

In [12]:
img_fp = '/data/UCSD_cxr/jpg/'
img_ids = glob('*.jpg', root_dir=img_fp)
img_ids.sort()
img_acc_ids = [int(re.search(r'[0-9]+',img_id)[0]) for img_id in img_ids]

In [13]:
acc_t2020 = rep_t2020[~rep_t2020['AccessionId'].str.contains('O')]

In [14]:
print('# Img Acc Ids: {}, # ids thru 2020: {}, # ids after 2020: {}, # img in thru 2020: {}, # img in after 2020: {}'.format( \
len(img_acc_ids), len(rep_t2020), len(rep_a2020), len(set(acc_t2020['AccessionId'].astype(int)).intersection(set(img_acc_ids))), len(set(rep_a2020['AccessionId'].astype(int)).intersection(set(img_acc_ids)))))

# Img Acc Ids: 112839, # ids thru 2020: 99536, # ids after 2020: 88852, # img in thru 2020: 44193, # img in after 2020: 61930


In [15]:
jpgs = glob('*.jpg', root_dir="/data/UCSD_cxr/jpg")

In [16]:
accn_ids = pd.Series([jpg.split('_')[1][:-4] for jpg in jpgs], name='accn_ids')

In [17]:
train_rep_a2020.merge(accn_ids, left_on='Accession', right_on='accn_ids').shape[0]

60762

In [18]:
train_rep_t2020.merge(accn_ids, left_on='Accession', right_on='accn_ids').shape[0]

43263